In [1]:
import pandas as pd
import os
import re
import random
import random
from deap import base, creator, gp, tools
import operator
from collections import defaultdict
import matplotlib.pyplot as plt
# import networkx as nx
# from networkx.drawing.nx_agraph import graphviz_layout
import pygraphviz as pgv

### GP setup

In [2]:
# Define the primitive set for a simple arithmetic expression
def protected_div(a, b):
    if abs(b) < 1e-6:
        return 1
    return a / b

pset = gp.PrimitiveSet("MAIN", 1)  # One input variable
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protected_div, 2)
pset.addPrimitive(pow, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))  # Random constants
pset.renameArguments(ARG0='x')

# Define fitness and individual classes
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax, pset=pset)

# Generate example individuals
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=2, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)


/home/nlplab/atwolin/EC/env_ec/lib/python3.12/site-packages/deap/gp.py:254: RuntimeWarning: Ephemeral rand101 function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "


In [3]:
__type__ = object

### Function

In [5]:
def print_tree(individual, file_name):
    nodes, edges, labels = gp.graph(individual)
    g = pgv.AGraph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    g.layout(prog="dot")

    for i in nodes:
        n = g.get_node(i)
        n.attr["label"] = labels[i]

    g.draw(f"{file_name}.pdf")

In [6]:
def combine_child(stack):
    while (len(stack[-1][1]) == stack[-1][0].arity and len(stack[-1][1]) < 2):
        # print(f"len(stack[-1][1]={len(stack[-1][1])}, stack[-1][0].arity={stack[-1][0].arity}")
        # Extract child
        prim, args, i2 = stack.pop()
        string = prim.format(*args)
        if len(stack) == 0:
            break
        # Add to its parent
        stack[-1][1].append(string)
        # print(f"[UPDATE] append stack: {stack[-1][1]}")
        # print(f"stack: {stack}")

In [37]:
def traverse_tree(stack, res, parent, idx):
    while (res != 0):
        # arity1 += 1
        # print(f"arity1: {arity1}")

        res -= 1
        # print(f"[WHILE -1] res: {res}")

        idx += 1
        stack.append((parent[idx], [], idx))
        # print(f"[WHILE] append stack: {parent[idx1].name}")
        res += parent[idx].arity
        # print(f"[WHILE +arity] res: {res}")

        # print("combine")
        combine_child(stack)

        # print(f"stack: {stack}")
    return stack, res, idx

In [51]:
def cxOnePoint(ind1, ind2):
    """Find the common region between two individuals and randomly select the crossover point as the root of the subtree in the region. Note that the subtree must be within the region.

    :param ind1: First tree participating in the crossover.
    :param ind2: Second tree participating in the crossover.
    :returns: A tuple of one tree.
    """
    idx1 = 0
    idx2 = 0

    # To track the trees
    stack1 = []
    stack2 = []

    # Store the common region
    region1 = []
    region2 = []

    # Start traversing the trees
    while (idx1 < len(ind1) and idx2 < len(ind2)):
        # Push the nodes to the stack
        # print("================================NEW================================")
        stack1.append((ind1[idx1], [], idx1))
        stack2.append((ind2[idx2], [], idx2))
        # print(f"append stack1: {ind1[idx1].name}")
        # print(f"append stack2: {ind2[idx2].name}")
        # print(f"stack1: {stack1}")
        # print(f"stack2: {stack2}")


        # Not the same region
        if (stack1[-1][0].arity != stack2[-1][0].arity):
            res1 = stack1[-1][0].arity
            res2 = stack2[-1][0].arity
            # print(f"res1: {res1}, res2: {res2}")
            # arity1 = 0  # number of child nodes of the current node
            # arity2 = 0
            stack1, res1, idx1 = traverse_tree(stack1, res1, ind1, idx1)
            # print("----------------------------------------STACK 2----------------------------------------")
            stack2, res2, idx2 = traverse_tree(stack2, res2, ind2, idx2)
        else:
            region1.append([ind1[idx1], idx1])
            region2.append([ind2[idx2], idx2])

        # print("-------------1 loop for combine------------")
        combine_child(stack1)
        combine_child(stack2)
        idx1 += 1
        idx2 += 1

    for pri, idx in region1:
        print(f"{idx}: {pri.name}")

    # Select crossover point
    point = random.randint(0, len(region1) - 1)
    print(f"crossover point: {point}")
    print(f"crossover point for trees: {region1[point]}, {region2[point]}")

    # Swap subtrees
    if (len(region1) > 0):
        slice1 = ind1.searchSubtree(region1[point][1])
        slice2 = ind2.searchSubtree(region2[point][1])
        ind1[slice1], ind2[slice2] = ind2[slice2], ind1[slice1]

    # Select the one has higher fitness value
    ### TODO ###

    return ind1, ind2

### Main

In [52]:
# Define the example trees (Parents)
# pare_str = "mul(mul(protected_div(x, x), pow(x)), sub(protected_div(x, x), x))"
# ind1_str = "mul(mul(x, pow(x)), sub(protected_div(x, x), x))"
# ind2_str = "protected_div(mul(pow(x), sub(x, -1)), mul(mul(1, 1), sub(1, 1)))"
ind1_str = "mul(mul(protected_div(x, sub(x, x)), sub(0, x)), sub(protected_div(x, x), sub(x, 1)))"
ind2_str = "protected_div(mul(x, x), mul(mul(1, 1), sub(1, 1)))"

ind1 = gp.PrimitiveTree.from_string(ind1_str, pset)
ind2 = gp.PrimitiveTree.from_string(ind2_str, pset)

print("Parent 1:", ind1)
print("Parent 2:", ind2)

Parent 1: mul(mul(protected_div(x, sub(x, x)), sub(0, x)), sub(protected_div(x, x), sub(x, 1)))
Parent 2: protected_div(mul(x, x), mul(mul(1, 1), sub(1, 1)))


In [53]:
ind1, ind2 = cxOnePoint(ind1, ind2)

0: mul
1: mul
10: sub
11: protected_div
12: ARG0
13: ARG0
14: sub
15: ARG0
16: 1
crossover point: 4
crossover point for trees: [<deap.gp.Terminal object at 0x7f9cbb754e40>, 12], [<deap.gp.Terminal object at 0x7f9cb986c700>, 6]


In [54]:
ind1.__str__(), ind2.__str__()

('mul(mul(protected_div(x, sub(x, x)), sub(0, x)), sub(protected_div(1, x), sub(x, 1)))',
 'protected_div(mul(x, x), mul(mul(x, 1), sub(1, 1)))')